# Yolo Evaluating

Here you can evaluate and make inferences with the yolass model

In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
from utils import init_notebook, benchmarking, predictions
%aimport datasets, utils.predictions, utils.benchmarking,utils.data_cleaning
from utils.predictions import * 

import sahi
from sahi import AutoDetectionModel
import fiftyone as fo
import torch

import os
from pathlib import Path

HOME = Path(os.getcwd()).parents[0]
HOME

print("torch available :",torch.cuda.is_available())
torch.cuda.empty_cache()

## Model importation

In [ ]:
model_name="yolass_aug_26ep"

yolo_detection_model = AutoDetectionModel.from_pretrained(
    model_type="yolov8",    # Model type (base model is yolov8 also for yolov10)
    model_path=f"../outputs/yolo/{model_name}.pt",    # Path to the model weights
    confidence_threshold=0.1,   # Confidence threshold
    # The higher the confidence threshold, the more precise they are (but we do a filtering later)
    device="cuda:0",  # to use the GPU
)

## Dataset importation

In [ ]:
# Importation of the dataset
dataset = fo.Dataset.from_dir(
    data_path=HOME/"data/coco_datasets/Cocass_aug/images",  # path to the dataset
    labels_path=HOME/"data/coco_datasets/Cocass_aug/fraw_detailed_val.json",   # path to the labels/annotations file
    dataset_type=fo.types.COCODetectionDataset, # the type of dataset to import
)
dataset

# the annotations are imported as "detections" but we want to rename them as "annotations" to avoid confusion
dataset.rename_sample_field("detections", "annotations")

In [ ]:
# To inspet the dataset on an external app
session = fo.launch_app(dataset, auto=False)
session.open_tab()

If the the dataset imported is made of crops (if not skip this kernel)

In [ ]:
kwargs={"slice_height":1280, "slice_width":1280, "overlap_height_ratio":0.9, "overlap_width_ratio":0.9}
for sample in dataset.iter_samples(progress=True, autosave=True):
    yolo_results=fo_predict_simple(yolo_detection_model,sample, 
        label_field="yolo_predictions", 
        kwargs=kwargs)

If the dataset is only a full page

In [ ]:
from utils.predictions import fo_predict_with_slicing
kwargs={"slice_height":1280, "slice_width":1280, "overlap_height_ratio":0.9, "overlap_width_ratio":0.9}
# We use Sliced Aided Hyper Inference to predict on the dataset
for sample in dataset.iter_samples(progress=True, autosave=True):
    yolo_results=fo_predict_with_slicing(yolo_detection_model,sample, 
        label_field="yolo_predictions", 
        slice_height=640, slice_width=640, overlap_height_ratio=0.8, overlap_width_ratio=0.8)

This part as been added to export the results in COCO format in json to filter them and return them on fiftyone

In [ ]:
from utils.data_cleaning import fiftyone_extraction_remapping,filter_annotations
import fiftyone.utils.coco as fouc
import json

# YOLO predictions filtering
# ------------------------
# Export the dataset to a JSON file
gt_json = HOME/"data/coco_datasets/Cocass_aug/fraw_detailed_val.json"
#gt_json = HOME/"data/coco_datasets/tests/Cassini_009_LoC/f009_detailed_updated.json"
dataset.export(
    export_dir="results/jsons/val/yolo_predictions",
    data_path=HOME/"data/coco_datasets/Cocass_aug/images",
    dataset_type=fo.types.COCODetectionDataset,  # You can choose other formats as well
    label_field="yolo_predictions",
    classes=dataset.get_classes('annotations'),
    export_media=False  # Set to True if you want to export media files as well
)
fiftyone_extraction_remapping(HOME/"notebooks/results/jsons/val/yolo_predictions/labels.json",
                              gt_json,inplace=True)
filter_annotations(json_file_path=HOME/"notebooks/results/jsons/val/yolo_predictions/labels.json",conf_threshold=0.5,
                    height_width_ratio= 0.3,
                    second_height_width_ratio=3,
                    iou_threshold= 0.8,inside_threshold=0.2,inplace=False,keep_annotations=True,class_agnostic=False,
                    keep_abbey=True)
fouc.add_coco_labels(
    dataset,
    "yolo_predictions_filtered",
    (HOME/"notebooks/results/jsons/val/yolo_predictions/filtered_labels.json").as_posix(),
    classes=dataset.get_classes('annotations'),
)

Evaluation of the model (we use the base fiftyone function)

In [ ]:
yolo_results = dataset.evaluate_detections(
    "yolo_predictions_filtered",
    gt_field="annotations",
    eval_key="yolo_eval",
    compute_mAP=True,
)

In [ ]:
print(" YOLO evaluation on test dataset :")
yolo_results.print_report(classes=dataset.get_classes('annotations'))
print("YOLO mAP :",yolo_results.mAP())

### Dataframes
This part can be used to export dataframes from the results (in csv, json or xlsx)

In [ ]:
from utils.benchmarking import fo_result2pd
yolo_df_9 = fo_result2pd(yolo_results,file_path="results/val/filtered_yolo_results.json",save_json=True)

## Confusion matrices
Those function takes as inputs the json exported from fiftyone. You may also use a built-in function from fiftyone but i find it hard to read and difficult to export.

In [ ]:
#I do not like the confusion matrix from fiftyone, I will use my own
from utils.benchmarking import fo_plot_confusion_matrix
plot = fo_plot_confusion_matrix(yolo_results,normalize=False)

In [ ]:
from utils.benchmarking import  build_confusion_matrix, plot_confusion_matrix
# Build the confusion matrix
confusion_matrix,categories = build_confusion_matrix(gt_json,pred_json)

In [ ]:
build_confusion_matrix(confusion_matrix,categories,normalize=True)